# Projeto Final - PFSP TT - Dupla 09

**Integrantes:**

* Lucas Hideki Takeuchi Okamura NUSP: 9274315

* Thales Arantes Kerche Nunes NUSP: 10769372

**Objetivo**

* Programar o algoritmo heurístico NEH, que busca minimizar o makespan em um flow shop com m máquinas

**Descrição**

O código roda o algoritmo de NEH, outro método para resolver o problema
de máquinas em série. O algoritmo está explicitado no artigo "A Heuristic 
Algorithm for the m-Machine, n-Job Flow-shop Sequencing Problem", 1983, por 
Nawaz, Muhammad; Jr, E Emory Enscore; Ham, Inyong. Resumidamente, tem-se 
os passos abaixo:

	* Passo 1: Ordenar a lista de jobs em ordem decrescente do tempo total de processamento;

	* Passo 2: Retire os dois primeiros da lista ordenada e crie uma nova lista, teste as duas permutações, escolha a
    permutação com menor makespan;

	* Passo 3: Retire o próximo job da lista ordenada, teste em todas as posições da nova lista, escolha a inserção
    com menor makespan;

	* Passo 4: Repita o passo 3 até esvaziar a lista ordenada; a nova lista resultante será a solução do problema

In [0]:
import xlwings as xl
import numpy as np

In [5]:
class Job:
    def __init__(self, i: int, pi: list, di: int):
        self.i=i     # número do job, pela ordem de chegada
        self.p=pi     # processing time
        self.p_total=sum(self.p)
        self.d=di     # due date
        self.C=0     # completion time
        self.T=0     # tardiness

In [0]:
def leInst(sheet_num: int) -> (int, int, list):
    pl1 = wb1.sheets[sheet_num]
    m=int(pl1.range('B5').value)     # converter em inteiro
    n=int(pl1.range('B6').value)     # converter em inteiro
    tab1=pl1.range('B11').options(expand='table', numbers=int).value     # ler como tabela e inteiros
    print()
    print('n =',n)
    print('m =',m)
    print('dados =',tab1)
    jobs=[]
    for i in range(n):
        pi=tab1[i][:-1]
        di=tab1[i][-1]
        jobs.append(Job(i,pi,di))

    return m,n,jobs

In [0]:
def gravaSched(i:int ,jobs: list,Cmax: int):
    n=len(jobs)
    m=len(jobs[0].p)
    tab=[]
    for job in jobs:
        a=[job.i+1]+job.p+job.C+job.d+job.T
        tab.append(a)

    if i==0:
        plan=wb2.sheets[0]
        plan.name='I(1)'
    else:
        plan=wb2.sheets.add('I('+str(i+1)+')',after=i)

    plan.range('A5').value=['m',m]
    plan.range('A6').value=['n',n]
    plan.range('E5').value=['Cmax',Cmax]
    plan.range('A10').value=['#','p1','p2', 'p3', 'p4', 'p5','C1','C2','C3','C4','C5', 'd', 'T']
    plan.range('A11').value=tab

In [0]:
def calcCmax(m: int,jobs: list) -> int:     # esta função calcula os tempos de conclusão dos jobs nas m máquinas
    n=len(jobs)
    c=[[0 for i in range(m)] for j in range(n)]
    p=[job.p for job in jobs]

    # primeiro job
    c[0][0]=p[0][0]     # primeira máquina
    for k in range(1,m):     # próximas máquinas
        c[0][k]=c[0][k-1]+p[0][k]

    # próximos jobs
    for j in range(1,n):
        c[j][0]=c[j-1][0]+p[j][0]     # primeira máquina
        for k in range(1,m):     # próximas máquinas
            c[j][k]=max(c[j][k-1],c[j-1][k])+p[j][k]

    j=0
    for job in jobs:
        job.C=c[j]
        j+=1

    return c[-1][-1]

def calculate_final_values(jobs: list) -> (dict, list):

    TT = nT = Fbar = Tmax = 0
    for job in jobs:
        job.T = max(0, job.C[-1] - job.d)
        TT += job.T
        Fbar += job.C[-1]
        if job.T > 0:
            nT += 1
            Tmax = max(Tmax, job.T)

    Cmax = jobs[-1].C[-1]

    Fbar = Fbar/len(jobs)

    return {'Cmax': Cmax, 'Fbar': Fbar, 'TT': TT, 'Tmax': Tmax, 'nT': nT}, jobs

def findBestPerm(jobs: list, job_insert: __main__.Job, m: int = 5) -> (list, int):
    minTardiness = np.inf
    totTardiness = 0

    for j in range(len(jobs) + 1):
        jobs.insert(j, job_insert)
        Cmax_test = calcCmax(m, jobs)
        for job in jobs:
            job.T = max(0, job.C[-1] - job.d)
            totTardiness += job.T
            if totTardiness < minTardiness:
                bestPos = j
                minTardiness = totTardiness
        jobs.pop(j)
    jobs.insert(bestPos, job_insert)

    return jobs, totTardiness

In [0]:
def schedNEHedd(jobs: list, m: int = 5):
    n = len(jobs)
    jobs.sort(key = lambda job: job.d, reverse = False)
    jobs1 = [jobs[0]]

    for k in range(2, n):

        findBestPerm(jobs, jobs[k])

In [0]:
def schedLS(jobs: list, bestTT: int, m: int = 5) -> (list, int):

    melhoria = True
    aux_jobs = jobs[:]
    while melhoria:
        melhoria = False
        jotas = list(range(len(aux_jobs)))
        random.shuffle(jotas)

        for j1 in jotas:
            job1 = aux_jobs.pop(j1)
            bestPos = j1

            for j2 in range(len(aux_jobs)+1):
                aux_jobs.insert(j2, job1)
                Cmax = calcCmax(m, aux_jobs)
                dados, aux_jobs = calculate_final_values(aux_jobs)

                if dados.TT < bestTT:
                    bestPos = j2
                    melhoria = True
                    bestTT = dados.TT
                    # break
                aux_jobs.pop(j2)
            # if melhoria:
            #     break
            aux_jobs.insert(bestPos, job1)


    return aux_jobs, dados.TT

In [0]:
def Temperatura(T: float, jobs: list, n: int = 20, m: int = 5) -> float:
    return (T*sum([job.p_total for job in jobs])/(10*n*m))

def findBestPosition(aux_jobs: list, new_job: Job, m: int = 5) -> (dict, list):

    bestTT = float('inf')
    for ind in range(len(aux_jobs)+1):
        aux_jobs.insert(ind, new_job)
        Cmax = calcCmax(m, aux_jobs)
        dados, aux_jobs = calculate_final_values(aux_jobs)

        if dados.TT < bestTT:
            bestPos = ind
            bestTT = dados.TT
        aux_jobs.pop(ind)
    aux_jobs.insert(bestPos, new_job)
    dados, aux_jobs = calculate_final_values(aux_jobs)

    return dados, aux_jobs


def schedIG(jobs: list, bestCmax: int, T: float = 0.5, d: int = 3, m: int = 5) -> (list, int):

    jobs_b = jobs[:]
    for _ in range(100):

        jobs_linha = jobs[:]
        retirados = random.sample(jobs_linha, d)
        for item in retirados:
            jobs_linha.remove(item)
        for new_job in retirados:
            dados, jobs_linha = findBestPosition(jobs_linha, new_job, m)

        if dados.TT < calculate_final_values(jobs)[0].TT or random.random() <= math.exp(-(dados.TT-calculate_final_values(jobs)[0].TT)/Temperatura(T, jobs)):
            jobs = jobs_linha[:]
#             if calcCmax(m, jobs) < calcCmax(m, jobs_b):
#                 jobs_b = jobs[:]
#         elif random.random() <= math.exp(-(newCmax-calcCmax(m, jobs))/Temperatura(T, jobs)):
#             jobs = jobs_linha[:]

    return jobs, calculate_final_values(jobs)